In [1]:
import pandas as pd
from statsmodels.regression.linear_model import OLS
import numpy as np

# 1. Load the data
data = pd.read_csv('backtest_data_1h.csv', parse_dates=True, index_col='time')

# 2. Calculate hedge ratio
def rolling_hedge_ratio(Y, X, window_size):
    hedge_ratios = []
    for i in range(len(Y) - window_size + 1):
        y = Y[i:i+window_size]
        x = X[i:i+window_size]
        model = OLS(y, x).fit()
        hedge_ratios.append(model.params[0])
    return pd.Series([np.nan] * (window_size - 1) + hedge_ratios, index=Y.index)

window = 90
data['hedge_ratio'] = rolling_hedge_ratio(data['xrp_close'], data['theta_close'], window)

# 3. Calculate the spread
data['calculated_spread'] = data['xrp_close'] - data['hedge_ratio'] * data['theta_close']

# 4. Calculate the rolling mean, standard deviation, and z-score
data['rolling_mean_spread'] = data['calculated_spread'].rolling(window=window).mean()
data['rolling_std_spread'] = data['calculated_spread'].rolling(window=window).std()
data['z_score'] = (data['calculated_spread'] - data['rolling_mean_spread']) / data['rolling_std_spread']

# 5. Determine trading signals
data['trading_signal'] = 0
data.loc[data['z_score'] < -1.8, 'trading_signal'] = 1  # Go long
data.loc[data['z_score'] > 1.8, 'trading_signal'] = -1  # Go short
data.loc[data['z_score'] * data['z_score'].shift(1) < 0, 'trading_signal'] = 0  # Go neutral

# 6. Backtest the strategy
data['strategy_returns'] = data['calculated_spread'].diff() * data['trading_signal'].shift(1)
data['cumulative_strategy_returns'] = (1 + data['strategy_returns']).cumprod()

print(data[['calculated_spread', 'trading_signal', 'strategy_returns', 'cumulative_strategy_returns']])

data.to_csv('backtest_result.csv', index=False)


                     calculated_spread  trading_signal  strategy_returns  \
time                                                                       
2023-05-30 16:30:00                NaN               0               NaN   
2023-05-30 17:30:00                NaN               0               NaN   
2023-05-30 18:30:00                NaN               0               NaN   
2023-05-30 19:30:00                NaN               0               NaN   
2023-05-30 20:30:00                NaN               0               NaN   
...                                ...             ...               ...   
2023-09-27 11:30:00           0.000331               0               0.0   
2023-09-27 12:30:00           0.000354               0               0.0   
2023-09-27 13:30:00           0.000377               0               0.0   
2023-09-27 14:30:00           0.000400               0               0.0   
2023-09-27 15:30:00           0.000509               0               0.0   

           